In [95]:
import arcpy
import requests
import io
import os
import zipfile

## Set filepaths - users will need to change data_path variable to a local file location

In [96]:
#data_path points all other paths to the right folder - I'm using and ArcGIS Pro project folder but any folder should work
data_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2"
gdb_path = os.path.join(data_path, "GIS5571_Lab2.gdb")

#set urls for data sources
lc_url = r"https://resources.gisdata.mn.gov/pub/gdrs/data/pub/edu_umn/base_landcover_minnesota/tif_base_landcover_minnesota.zip"
dem_url = r"https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip"

lc_res = requests.get(lc_url)
dem_res = requests.get(dem_url)

dem_path = os.path.join(data_path, "fgdb_elev_30m_digital_elevation_model.zip")
lc_path = os.path.join(data_path, "tif_base_landcover_minnesota.zip")

In [97]:
#save data to file
with open(dem_path, 'wb') as f:
    f.write(dem_res.content)
    
with open(lc_path, 'wb') as f:
    f.write(lc_res.content)

In [98]:
#set unzip paths
dem_unzip = os.path.join(data_path, "30m_dem")
lc_unzip = os.path.join(data_path, "tif_lc")

#unzip both datasets
with zipfile.ZipFile(dem_path, 'r') as zip_ref:
    zip_ref.extractall(dem_unzip)
    
with zipfile.ZipFile(lc_path, 'r') as zip_ref:
    zip_ref.extractall(lc_unzip)
    


In [99]:
#I'm going to give Dory 3 miles around the start and ending points as potential places to travel - so first I'll create buffers

arcpy.analysis.PairwiseBuffer(
    in_features="start_end_pts",
    out_feature_class= os.path.join(gdb_path, "start_end_pts_buffer"),
    buffer_distance_or_field="3 Miles",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR",
    max_deviation="0 Meters"
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\GIS5571_Lab2.gdb\\start_end_pts_buffer'>

In [103]:
#I'll then get the extent of those buffers to use as a parameter for defining the study area
extent = arcpy.Describe(os.path.join(gdb_path, "start_end_pts_buffer")).extent

#use that extent with the extract by rectangle tool to get my clipped dem
with arcpy.EnvManager(scratchWorkspace= gdb_path):
    clipped_dem = arcpy.sa.ExtractByRectangle(
        in_raster= os.path.join(dem_unzip, "elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m"),
        extent= extent,
        extraction_area="INSIDE"
    )
    clipped_dem.save(os.path.join(gdb_path, "clipped_dem"))
    
#use that same extent to get a clipped land use raster too
with arcpy.EnvManager(scratchWorkspace= gdb_path):
    clipped_lc = arcpy.sa.ExtractByRectangle(
        in_raster= os.path.join(lc_unzip, "landcover_impervious_statewide2013_v2.tif"),
        extent= extent,
        extraction_area="INSIDE"
    )
    clipped_lc.save(os.path.join(gdb_path, "clipped_lc"))

In [104]:
#run slope tool to convert dem to slope
arcpy.ddd.Slope(
    in_raster="clipped_dem",
    out_raster= os.path.join(gdb_path, "slope_surface"),
    output_measurement="DEGREE",
    z_factor=1,
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\GIS5571_Lab2.gdb\\slope_surface'>

In [105]:
#Next I'm going to resample the dem so it has the same spatial resolution as the land cover raster
arcpy.management.Resample(
    in_raster="slope_surface",
    out_raster= os.path.join(gdb_path, "resample_slope"),
    cell_size="15 15",
    resampling_type="BILINEAR"
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\GIS5571_Lab2.gdb\\resample_slope'>

In [106]:
#Next, we resample the rasters and assign values based on slope and landclass that will represent the cost to cross these pixels

#Slope - heavily favors lightly sloped land
reclass_slope = arcpy.ddd.Reclassify(
    in_raster="resample_slope",
    reclass_field="VALUE",
    remap="0 3 1;3 6 2;6 10 3;10 20 4;20 90 5",
    out_raster= os.path.join(gdb_path, "Reclass_slope"),
    missing_values="DATA"
)


#Land use - we know Dory doesn't like crossing farm fields, so those categories get a 5 value
#She also doesn't like crossing water if she can help it so open water and wetlands get classed 4
#urbanized areas and roadways, managed grass, and forested areas get 1, 2 and 3 values, respectively
reclass_lc = arcpy.ddd.Reclassify(
    in_raster="clipped_lc",
    reclass_field="Value",
    remap="1 100 1;101 102 3;103 4;104 107 2;108 2;109 110 5",
    out_raster= os.path.join(gdb_path, "Reclass_lc"),
    missing_values="DATA"
)

In [108]:
#define list of weights for first raster (initially I had five weights but it was just taking a long time so I did 3 instead)
weights = [.3, .5, .7]
#define empty list of weighted cost surface names to go in - we'll use this in the next step to generate cost paths
wcs_list = []

#redefine reclassed variables from above as rasters so we can do raster algebra to them
reclass_lc = arcpy.Raster(reclass_lc)
reclass_slope = arcpy.Raster(reclass_slope)

#for loop to create a new weighted cost surface for each of the weighting schemes 
for weight in weights:
    # Define the output path, including the weight values in the filenames
    wcs_name = f"wcs_lc{int(weight * 100)}_sl{int((1-weight)*100)}"
    output_path =  os.path.join(gdb_path, f"{wcs_name}")
    
    # Perform the raster calculation
    wcs = (weight * reclass_lc) + ((1 - weight) * reclass_slope)
    
    # Save the output raster with the unique name
    wcs.save(output_path)
    
    # Add the raster name to the list
    wcs_list.append(wcs_name)
    
    print(f"Output raster saved at {output_path}")

# Print the list of raster names
print("Created weighted cost surface names:", wcs_list)

Output raster saved at C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb\wcs_lc30_sl70
Output raster saved at C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb\wcs_lc50_sl50
Output raster saved at C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb\wcs_lc70_sl30
Created weighted cost surface names: ['wcs_lc30_sl70', 'wcs_lc50_sl50', 'wcs_lc70_sl30']


In [109]:
for wcs_name in wcs_list:
    # Define the output feature class path, incorporating the raster name to make it unique
    out_line = f"{gdb_path}\\Optimal_{wcs_name}"
    
    # Run the OptimalRegionConnections tool with the current raster as the cost raster
    arcpy.sa.OptimalRegionConnections(
        in_regions="start_end_pts",
        out_feature_class= out_line,
        in_barrier_data=None,
        in_cost_raster=wcs_name,
        out_neighbor_paths=None,
        distance_method="PLANAR",
        connections_within_regions="GENERATE_CONNECTIONS"
    )
    
    print(f"{wcs_name} optimal route complete. Saved at {out_line}")

wcs_lc30_sl70 optimal route complete. Saved at C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb\Optimal_wcs_lc30_sl70
wcs_lc50_sl50 optimal route complete. Saved at C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb\Optimal_wcs_lc50_sl50
wcs_lc70_sl30 optimal route complete. Saved at C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb\Optimal_wcs_lc70_sl30
